In [ ]:
# default_exp workflows
import collections
import copy
import json
import logging
from dataclasses import dataclass
from enum import Enum

from functools import partial
from typing import Set, Mapping

from traitlets import HasTraits
from dharpa_toolbox.modules.core import DharpaModule

from dharpa_toolbox.utils import get_subclass_map, get_module_name_from_class, to_camel_case
import networkx as nx
import typing
#export

log = logging.getLogger("dharpa-toolbox")

modules_to_load = ["dharpa_toolbox.modules.core", "dharpa_toolbox.modules.files", "dharpa_toolbox.modules.text"]

ALL_MODULE_CLASSES = get_subclass_map(DharpaModule, preload_modules=modules_to_load, key_func=get_module_name_from_class)


# @dataclass
# class InputOutputConnection(object):
#
#     input_module: DharpaModule
#     input_value: str
#     output_module: DharpaModule
#     output_value: str
#
#     def __hash__(self):
#
#         return hash(self.input_module, self.input_value, self.output_module, self.output_value)
#
#     def __eq__(self, other):
#
#         if self.__class__ != other.__class__:
#             return False
#
#         return (self.input_module, self.input_value, self.output_module, self.output_value) == (other.input_module, other.input_value, other.output_module, other.output_value)

class ValueLocationType(Enum):
    workflow_input = 0
    workflow_output = 1
    module_input = 2
    module_output = 3

class ValueLocation(typing.NamedTuple):
    module: DharpaModule
    value_name: str
    type: ValueLocationType

    def __hash__(self):

        return hash((self.module, self.value_name, self.type))

    def __eq__(self, other):

        if not isinstance(other, ValueLocation):
            return False

        return (self.module, self.value_name, self.type) == (other.module, other.value_name, other.type)

    def __str__(self):

        if self.type == ValueLocationType.module_output:
            t = f"{self.module.id} output"
        elif self.type == ValueLocationType.workflow_input:
            t = "user_input"
        elif self.type == ValueLocationType.workflow_output:
            t = "result"
        else:
            t = f"{self.module.id} input"

        # return f"{self.module.id}.{t}.{self.value_name}"
        return f"{t}: {self.value_name}"
        # return f"{self.module.id} {t}: {self.value_name}"


class DharpaWorkflow(DharpaModule):

    def __init__(self, **config: typing.Any):

        self._module_details: Mapping[str, Mapping[str, typing.Any]] = {}

        self._execution_graph: nx.DiGraph = None
        self._data_flow_graph: nx.DiGraph = None

        self._workflow_inputs: Mapping = None
        self._workflow_outputs: Mapping = None
        self._execution_stages: typing.Iterable[typing.Iterable[str]] = None

        super().__init__(**config)

    def _input_updated(self, change) -> typing.Any:

        log.debug(f"Input updated for workflow ({self.id}): {change}")

        if change.name not in self._input_staging.keys():
            self._input_staging[change.name] = {"old": change.old, "new": change.new}
        else:
            self._input_staging[change.name]["new"] = change.new

        module_obj, input_name = self._workflow_inputs[change.name]

        trait = module_obj.inputs.traits().get(input_name)
        trait.set(module_obj.inputs, change.new)

        self._check_stale()

    def _output_udpated(self, output_name: str, change):

        log.debug(f"Workflow output '{output_name}' updated: {change.new}")
        print(f"Workflow output '{output_name}' updated: {change.new}")


    def _preprocess_config(self, **config: typing.Any):

        modules = config.get("modules", None)
        if not modules:
            raise ValueError("Can't create workflow: no modules specified")

        no_modules_yet = len(self._module_details) == 0

        # TODO: re-use already existing modules
        self._module_details.clear()
        self._workflow_inputs = {}
        self._workflow_outputs = {}

        self._execution_graph = nx.DiGraph()
        self._data_flow_graph = nx.DiGraph()

        self._execution_stages = []

        module_ids = set()

        all_outputs_map: Mapping[str, ValueLocation] = {}

        for m in modules:

            module_type = m["type"]
            module_config = m.get("config", {})
            module_id = m.get("id", None)
            module_input_map = m.get("input_map", {})

            if isinstance(module_type, str):
                module_cls = ALL_MODULE_CLASSES.get(module_type)
            elif isinstance(module_type, type):
                module_cls = module_type
            else:
                raise TypeError(f"Invalid class for module type: {type(module_type)}")

            if module_id is None:
                if not no_modules_yet:
                    raise Exception(f"Module config without 'id' property not allowed after initial creation of workflow: {m}")
                module_obj = module_cls(**module_config)
                module_id = module_obj.id
                self._module_details[module_id] = {
                    "module": module_obj,
                }
                m["id"] = module_id
            elif module_id not in self._module_details.keys():
                if module_id in module_ids:
                    raise Exception(f"Duplicate module id: {module_id}")
                module_obj = module_cls(id=module_id, **module_config)
                self._module_details[module_id] = {
                    "module": module_obj,
                }
            else:
                if module_id in module_ids:
                    raise Exception(f"Duplicate module id: {module_id}")

                module_obj = self._module_details[module_id]["module"]
                module_obj.set_config(module_config)
                # TODO: remove current input map

            module_ids.add(module_id)

            # creating a map of all inputs and their connections
            full_input_map = {}
            for name in module_obj.inputs.trait_names():
                mapped_input = module_input_map.get(name, None)
                if not mapped_input:
                    output_name = f"{module_id}__{name}"
                    wi = ValueLocation(module=self, value_name=output_name, type=ValueLocationType.workflow_input)
                    full_input_map[name] = wi
                    vl = ValueLocation(module=module_obj, value_name=name, type=ValueLocationType.module_input)
                    self._workflow_inputs[output_name] = vl
                    all_outputs_map.setdefault(self.id, {})[output_name] = vl
                    self.data_flow_graph.add_edge(wi, vl)
                    # self.data_flow_graph.add_edge(self, wi)
                    self.data_flow_graph.add_edge(vl, module_obj)
                else:
                    if isinstance(mapped_input, str):
                        o1 = ValueLocation(module=self.get_module(mapped_input), value_name=name, type=ValueLocationType.module_output)
                        full_input_map[name] = o1
                        i1 = ValueLocation(module=module_obj, value_name=name, type=ValueLocationType.module_input)
                        all_outputs_map.setdefault(mapped_input, {})[name] = i1
                    elif isinstance(mapped_input, collections.abc.Iterable) and len(mapped_input) == 2:
                        # TODO: check format of mapped_input
                        o1 = ValueLocation(module=self.get_module(mapped_input[0]), value_name=mapped_input[1], type=ValueLocationType.module_output)
                        full_input_map[name] = o1
                        i1 = ValueLocation(module=module_obj, value_name=name, type=ValueLocationType.module_input)
                        all_outputs_map.setdefault(mapped_input[0], {})[mapped_input[1]] = i1

                    else:
                        raise ValueError(f"Invalid value for input mapping: {mapped_input}")

                    self.data_flow_graph.add_edge(o1.module, o1)
                    self.data_flow_graph.add_edge(o1, i1)
                    self.data_flow_graph.add_edge(i1, module_obj)

            self._module_details[module_id]["input_map"] = full_input_map

            input_locations: typing.Iterable[ValueLocation] = full_input_map.values()

            if not input_locations or all(x.type == ValueLocationType.workflow_input for x in input_locations):
                self._execution_graph.add_edge(self, module_obj)

            for loc in input_locations:
                if loc.type != ValueLocationType.workflow_input:
                    self._execution_graph.add_edge(loc.module, module_obj)

        for module_id, module_details in self._module_details.items():
            module = module_details["module"]
            self._module_details[module_id]["output_map"] = all_outputs_map.get(module_id, {})
            for output in module.outputs.trait_names():
                output_name = f"{module_id}__{output}"
                mo = ValueLocation(module=module, value_name=output, type=ValueLocationType.module_output)
                self._workflow_outputs[output_name] = mo
                wo = ValueLocation(module=self, value_name=output_name, type=ValueLocationType.workflow_output)
                self.data_flow_graph.add_edge(module, mo)
                self.data_flow_graph.add_edge(mo, wo)

        # calculate execution order
        path_lengths = {}
        for m_d in self._module_details.values():

            m = m_d["module"]

            paths = list(nx.all_simple_paths(self.execution_graph, self, m))
            max_steps = max(paths, key=lambda x: len(x))
            path_lengths[m] = len(max_steps)-1

        max_length = max(path_lengths.values())

        for i in range(1, max_length+1):
            stage = [m for m, l in path_lengths.items() if l == i]
            self._execution_stages.append(stage)
            for m in stage:
                self._module_details[m.id]["processing_stage"] = i

        return config

    @property
    def execution_graph(self) -> nx.DiGraph:

        return self._execution_graph

    @property
    def data_flow_graph(self) -> nx.DiGraph:

        return self._data_flow_graph


    def _create_inputs(self, **config) -> HasTraits:

        traits = {}

        for name, module_input in self._workflow_inputs.items():

            m: DharpaModule = module_input[0]
            trait_name = module_input[1]
            trait = m.inputs.traits().get(trait_name)

            traits[name] = copy.deepcopy(trait)

        inputs_cls = type(f"WorkflowInputValues{to_camel_case(self.id, capitalize=True)}", (HasTraits,), traits)
        return inputs_cls()

    def _create_outputs(self, **config) -> HasTraits:

        traits = {}

        for module_name, module in self.modules.items():

            for output_name in module.outputs.trait_names():
                trait = module.outputs.traits().get(output_name)
                output_trait_name = f"{module_name}__{output_name}"
                traits[output_trait_name] = copy.deepcopy(trait)
                update_func = partial(self._output_udpated, output_trait_name)
                module.outputs.observe(update_func, names=output_name)

        outputs_cls = type(f"WorkflowOutputValues{to_camel_case(self.id, capitalize=True)}", (HasTraits,), traits)
        return outputs_cls()

    @property
    def modules(self) -> typing.Mapping[str, DharpaModule]:
        return {m_name: m_details["module"] for m_name, m_details in self._module_details.items()}

    def get_module(self, id: str) -> DharpaModule:

        md = self._module_details.get(id, None)
        if md is None:
            raise Exception(f"No module '{id}' in workflow '{self.id}'.")

        return md["module"]

    @property
    def module_ids(self) -> typing.Iterable[str]:

        return self._module_details.keys()

    def _check_stale(self):

        for m in self._module_details.values():
            if m["module"].stale:
                self._state.stale = True
                return True

        self._state.stale = False
        return False

    # def _module_input_updated(self, source_module: DharpaModule, source_input_name: str, change):
    #
    #     print(f"MODULE INPUT UPDATED: {source_module} - {source_input_name}")
    #
    #     # raise Exception(change)
    #
    #     # print("-------------------")
    #     # print(f"MODULE INPUT UPDATED: {source_module}")
    #     # print(f"INPUT NAME: {source_input_name}")
    #     # # print(change)
    #     # # print(change.new)
    #     # print("-------------------")
    #     self._state.stale = True
    #     # deps = self.dependencies.get(source_module.id)
    #     # print(f"Dependencies: {self.dependencies.get(source_module.id)}")
    #     # for d in deps:
    #     #     dep_module = self.get_module(d)
    #     #     print(dep_module.input_mapping)
    #
    #     # source_module.process()
    #
    #
    # def _module_output_updated(self, source_module: DharpaModule, source_output_name: str, change):
    #
    #     print(f"MODULE OUTPUT UPDATED: {source_module} - {source_output_name}")

    def _process(self, **inputs) -> Mapping[str, typing.Any]:

        modules_executed = set()

        for i, modules_to_execute in enumerate(self._execution_stages):

            print(f"Executing level: {i+1}")
            for m in modules_to_execute:
                print(f"Executing: {m}")
                m.process()

                for output_name in m.outputs.trait_names():
                    print(f"Setting output: {output_name}")
                    value = getattr(m.outputs, output_name)
                    print(f"Value: {value}")
                    value_location = ValueLocation(module=m, value_name=output_name, type=ValueLocationType.module_output)
                    desc = self.data_flow_graph.out_edges(value_location)
                    print(f"Descendants:")
                    for d in desc:
                        target_loc = d[1]
                        if target_loc.type == ValueLocationType.module_input:
                            target_module: DharpaModule = target_loc.module
                            target_value_name: str = target_loc.value_name
                            target_module.inputs.set_trait(target_value_name, value)
                            print(f"TARGET: {target_loc}")
                print("-------")

            modules_executed.update(modules_to_execute)

            # for m in modules_to_execute:
            #     deps = self._dependencies.get(m)
            #     for d in deps:
            #         if d in modules_executed:
            #             raise Exception(f"Can't set dependency value from {m} to {d}: module {d} already executed")
            #         print(f"ADDING INPUT FROM {m} TO {d}")

        # print("Executing workflow")
        # print("----------------")
        # print("dependencies:")
        # print(self._dependencies)
        # print("----------------")
        # print("dependencies reverse:")
        # print(self._dependencies_reverse)
        #
        # print(self.modules)





        self._check_stale()
        self.busy = False


    # def add_module(self, module: DharpaModule):
    #
    #     if self._state.initialized:
    #         raise Exception(f"Can't add module '{module.id}': workflow already initialized")
    #     self.modules.append(module)
    #     self._state.stale = True
    #
    # def add_modules(self, *modules: DharpaModule):
    #
    #     for module in modules:
    #         self.add_module(module)
    #
    # def get_module(self, module_id: str) -> Optional[DharpaModule]:
    #
    #     result = None
    #     for m in self.modules:
    #         if m.id == module_id:
    #             result = m
    #             break
    #
    #     if result is None:
    #         raise Exception(f"Worfklow does not have module with id {module_id}.")
    #     return result

    @property
    def current_state(self):

        result = {"modules": {}}
        for module_id, module in self.modules.items():
            result["modules"][module_id] = module.current_state
        result["stale"] = self._state.stale
        return result

    @property
    def current_structure(self) -> typing.Dict[str, typing.Any]:

        result = []
        for module_details in self._module_details.values():
            module = module_details["module"]
            processing_stage = module_details["processing_stage"]
            module_dict = {"module_id": module.id, "processing_stage": processing_stage}
            input_map = module_details["input_map"]
            output_map = module_details["output_map"]
            module_dict["inputs"] = {}
            for input_name, target in input_map.items():
                if target.module == self:
                    module_dict["inputs"][input_name] = {"user_input": True}
                else:
                    target_module_id = target.module.id
                    target_value_name = target.value_name
                    module_dict["inputs"][input_name] = {"module_id": target_module_id, "output_name": target_value_name, "user_input": False}
            for output_name, source in output_map.items():
                module_dict["output_names"] = list(output_map.keys())

            result.append(module_dict)

        return result



    def __repr__(self) -> str:
        return f"{self.__class__.__name__}(id='{self.id}' input_names={self.inputs.trait_names()} output_names={self.outputs.trait_names()}) modules={self.modules} config={self._config_raw}"

    def __str__(self):

        return f"workflow: '{self.id}'"

In [ ]:
from dharpa_toolbox.core import export_notebooks
export_notebooks()

Converted 00_core.ipynb.
Converted 01_data_types.ipynb.
Converted 02_modules_core.ipynb.
Converted 03_data_sources.ipynb.
Converted 03_data_targets.ipynb.
Converted 05_workflows.ipynb.
Converted 10_modules_files.ipynb.
Converted 10_modules_text.ipynb.
Converted 99_utils.ipynb.
Converted index.ipynb.
